In [ ]:
import pandas as pd
import numpy as np
from os import path

In [15]:
weather_train = pd.read_csv('../input/ashrae-energy-prediction/weather_train.csv')
weather_test = pd.read_csv('../input/ashrae-energy-prediction/weather_test.csv')


In [16]:
def fancyBackfilling(df):
    #do fancy stuff
    return df

In [24]:
# https://www.kaggle.com/nz0722/aligned-timestamp-lgbm-by-meter-type
def alignTimePickle():
    weather = pd.concat([weather_train,weather_test],ignore_index=True)
    weather['timestamp'] = pd.to_datetime(weather['timestamp'])
    weather_key = ['site_id', 'timestamp']

    temp_skeleton = weather[weather_key + ['air_temperature']].drop_duplicates(subset=weather_key).sort_values(by=weather_key).copy()

    # calculate ranks of hourly temperatures within date/site_id chunks
    temp_skeleton['temp_rank'] = temp_skeleton.groupby(['site_id', temp_skeleton.timestamp.dt.date])['air_temperature'].rank('average')

    # create a dataframe of site_ids (0-16) x mean hour rank of temperature within day (0-23)
    df_2d = temp_skeleton.groupby(['site_id', temp_skeleton.timestamp.dt.hour])['temp_rank'].mean().unstack(level=1)

    # Subtract the columnID of temperature peak by 14, getting the timestamp alignment gap.
    site_ids_offsets = pd.Series(df_2d.values.argmax(axis=1) - 14)
    site_ids_offsets.index.name = 'site_id'
    site_ids_offsets.to_pickle('../input/ashrae-energy-prediction/site_time_offsets_df.pickle')
    print(site_ids_offsets)

alignTimePickle()

site_id
0     5
1     0
2     9
3     6
4     8
5     0
6     6
7     6
8     5
9     7
10    8
11    6
12    0
13    7
14    6
15    6
dtype: int64


In [17]:
weather_train = fancyBackfilling(weather_train);
weather_test = fancyBackfilling(weather_test);

In [18]:
# savve modified files in _ directory
if not path.exists('../input/_ashrae-energy-prediction'):
     !mkdir -p ../input/_ashrae-energy-prediction
    
weather_train.to_csv('../input/_ashrae-energy-prediction/weather_train.csv')
weather_test.to_csv('../input/_ashrae-energy-prediction/weather_test.csv')
